<a href="https://colab.research.google.com/github/rohith033/BTP/blob/main/dqn_v.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'shimmy'
!pip install stable-baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

In [ ]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback

In [ ]:
from osgeo import gdal
import numpy as np

# Load the raster file
raster = gdal.Open('/content/Depth (Max) (1).Terrain.needed_tif.tif')
band = raster.GetRasterBand(1)  # Assuming depth is in band 1
depth_data = band.ReadAsArray()

# Get geotransformation
geotransform = raster.GetGeoTransform()

In [ ]:
def get_flat_index_neighbors_with_diagonals(flat_index, num_rows, num_cols):
    i = flat_index // num_cols
    j = flat_index % num_cols
    neighbors = {}

    # Direct neighbors
    if i > 0 and (i-1) * num_cols + j<num_rows*num_cols:
        neighbors['top'] = (i-1) * num_cols + j
    if i < num_rows - 1 and (i+1) * num_cols + j<num_rows*num_cols:
        neighbors['bottom'] = (i+1) * num_cols + j
    if j > 0 and i * num_cols + (j-1) < num_rows*num_cols :
        neighbors['left'] = i * num_cols + (j-1)
    if j < num_cols - 1 and i * num_cols + (j+1) < num_rows*num_cols:
        neighbors['right'] = i * num_cols + (j+1)

    # Diagonal neighbors
    if i > 0 and j > 0 and (i-1) * num_cols + (j-1) < num_rows*num_cols:
        neighbors['top_left'] = (i-1) * num_cols + (j-1)
    if i > 0 and j < num_cols - 1 and (i-1) * num_cols + (j+1) <num_rows*num_cols:
        neighbors['top_right'] = (i-1) * num_cols + (j+1)
    if i < num_rows - 1 and j > 0 and (i+1) * num_cols + (j-1) < num_rows*num_cols:
        neighbors['bottom_left'] = (i+1) * num_cols + (j-1)
    if i < num_rows - 1 and j < num_cols - 1 and (i+1) * num_cols + (j+1)<num_rows*num_cols:
        neighbors['bottom_right'] = (i+1) * num_cols + (j+1)

    return neighbors

In [ ]:
num_rows, num_cols = depth_data.shape
lats = np.array([geotransform[3] + geotransform[5] * i for i in range(num_rows)])
longs = np.array([geotransform[0] + geotransform[1] * j for j in range(num_cols)])
# Create an empty list to store the data
data_matrix = []

# Iterate through each cell
for i in range(num_rows):
    for j in range(num_cols):
        lat = lats[i]
        lon = longs[j]
        depth = depth_data[i, j]
        data_matrix.append([lat, lon, depth])

# Optionally convert to a numpy array for better handling
data_matrix = np.array(data_matrix)
data_matrix[data_matrix == -9999] = 0

In [ ]:
total_states =  4000
environments = []

In [ ]:
def make_env(flat_index,base_env,i,j):
  if i<0 or j<0 or i>8 or j>8 or base_env[i][j] != 0:
    return
  base_env[i][j] = data_matrix[flat_index][2]
  if base_env[i][j] == 0:
    base_env[i][j]  = 0.0001
  space = get_flat_index_neighbors_with_diagonals(flat_index,num_rows,num_cols)
  if 'top' in space:
    make_env(space['top'],base_env,i-1,j)
  if 'bottom' in space:
    make_env(space['bottom'],base_env,i+1,j)
  if 'left' in space:
    make_env(space['left'],base_env,i,j-1)
  if 'right' in space:
    make_env(space['right'],base_env,i,j+1)
  return

In [ ]:
for flat_index , depth_info in enumerate(data_matrix):
  if total_states == 0:
    break
  if depth_info[2] > 0.99:
    total_states-=1
    base_env = np.zeros((9,9))
    make_env(flat_index,base_env,4,4)
    environments.append(base_env)

In [43]:
land_data =  [
            "CCCFFCCCC",
             "CFCFCFFFC",
             "CFCCCFHCC",
             "CFCCCFHCC",
             "CCCFFCCCC",
             "CFCCCFHCC",
             "CFFCFBHCC",
             "CFFFFFCCC",
             "CFFFCCCCC"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
class CustomFrozenLakeEnv(gym.Env):
    def __init__(self,depth_data,land_data):
        self.desc = land_data

        self.visited = set()
        # Generate random depths between 0.5 and 5 for 'C' blocks
        self.depth_map = np.array(depth_data)
        self.colm = self.depth_map[0].shape[0]
        self.nS = 81
        self.nA = 4  # Define the number of actions (left, down, right, up)
        self.start_state = self._get_state(row=4, col=4)
        min_depth_point = np.unravel_index(np.argmin(self.depth_map), self.depth_map.shape)
        self.thersold = 0.2
        self.goal_state = self._get_state(row=min_depth_point[0], col=min_depth_point[1])
        self.state = None
        self.action_space = gym.spaces.Discrete(self.nA)  # Define the action space
        self.observation_space = gym.spaces.Discrete(self.nS)  # Define the observation space
        self.reset()

    def _get_state(self, row, col):
        """Converts row and column coordinates to a state index."""
        return row * 9 + col

    def reset(self):
        self.state = self.start_state
        self.visited = set()
        return self._get_obs(self.state)

    def _get_obs(self, state):
        """Returns the observation (state index) for the current state."""
        return state

    def step(self, action):
        state = self.state
        reward = 0
        done = False
        # Probabilistic movement (same as standard FrozenLake)
        if np.random.rand() < 0.1:
            state = (state + np.random.randint(-1, 2, size=1)) % self.nS
        else:
            new_state = state
            if action == 0:
              new_state-=1
            if action == 1:
              new_state+=1
            if action == 2:
              new_state = (state)*(int((state/9))-1) + state%9
            if action == 3:
              new_state = (state)*(int((state/9))+1) + state%9
            if 0 <= new_state < self.nS:
                state = new_state
        if 0 > state or state>=self.nS:
            reward = -1e5
            done = True
            return self._get_obs(state), -1e5 , done, {}
        if int(state) in self.visited:
            reward = -1e5
            done = True
            return self._get_obs(state), -1e5 , done, {}
        self.visited.add(int(state))
        # Update depth reward
        desc = np.array(list(''.join(self.desc))).reshape(9,9)
        crow = int(state / self.colm)
        ccol =  state % self.colm
        reward += -(((self.depth_map[crow,ccol]+1)/10)*reward)
        if desc[crow][ccol] == 'F':
          reward += -(reward*0.1)
        if desc[crow][ccol] == 'B':
          reward += -(reward*0.1)
        if self.depth_map[crow,ccol] <= self.thersold:
          done = True
        # Check for terminal states (hole or goal)
        if state == self.goal_state:
            done = True
        self.state = state
        return self._get_obs(state), reward, done, {}
    def render(self, mode='human'):
        """Renders a text-based representation of the environment."""
        if mode == 'human':
            desc = np.array(list(''.join(self.desc))).reshape(9,9)
            for i in range(9):
                for j in range(9):
                    if self.state == self._get_state(i, j):
                        desc[i, j] = desc[i, j].lower()  # Highlight current state
                print(''.join(desc[i]))
        else:
            super().render(mode=mode)  # Default rendering for compatibility (optional)
# Example usage
env = CustomFrozenLakeEnv(environments[999],land_data)
state = env.reset()
done = False
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
# Save the trained model
model.save("trained_dqn")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.5      |
|    ep_rew_mean      | -1e+05   |
|    exploration_rate | 0.991    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4917     |
|    time_elapsed     | 0        |
|    total_timesteps  | 10       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.75     |
|    ep_rew_mean      | -1e+05   |
|    exploration_rate | 0.979    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2333     |
|    time_elapsed     | 0        |
|    total_timesteps  | 22       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.58     |
|    ep_rew_mean      | -1e+05 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
<ipython-input-53-16e60f310ad9>:56: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if int(state) in self.visited:
/usr/l

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.67     |
|    ep_rew_mean      | -1e+05   |
|    exploration_rate | 0.818    |
| time/               |          |
|    episodes         | 72       |
|    fps              | 1098     |
|    time_elapsed     | 0        |
|    total_timesteps  | 192      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 5e+04    |
|    n_updates        | 22       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.66     |
|    ep_rew_mean      | -1e+05   |
|    exploration_rate | 0.808    |
| time/               |          |
|    episodes         | 76       |
|    fps              | 1065     |
|    time_elapsed     | 0        |
|    total_timesteps  | 202      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 4.37e+04 |
|    n_updates      

In [ ]:
# prompt: 9 * 9 numpy array with values from 0.2 to 5

import numpy as np

# Create a 9x9 numpy array with values from 0.2 to 5
array = np.random.uniform(low=0.2, high=5, size=(9, 9))

# Print the array
print(array)


[[2.14882363 3.81925245 3.49198886 1.39574906 2.20659804 3.31098253
  2.13414789 1.02483365 4.26170452]
 [3.75439336 4.87402717 2.80059153 4.81237387 2.77980385 3.51839146
  4.01257913 0.24306444 1.57716522]
 [1.21771757 4.43570744 0.66256174 4.14364684 1.43020257 2.89611853
  3.19947347 2.70350746 2.38354314]
 [3.76664735 4.33081333 2.27269869 0.24621764 4.13989828 4.78571449
  0.68529991 4.59654571 2.49546005]
 [3.76714587 3.34873027 3.53145313 2.43241247 2.36928826 2.66905926
  3.84524324 2.85609766 2.29151511]
 [3.30414203 0.2350408  4.77503466 2.80682417 4.39052741 0.32393449
  1.95595678 0.35813213 2.64172655]
 [2.55768324 4.14954048 1.97322506 0.43026542 2.8858188  3.84575875
  1.52798385 4.59289047 4.79442063]
 [1.49359886 4.52719658 4.62812229 0.92871785 1.10921065 0.48148187
  3.08501161 4.02371696 1.33236533]
 [3.013228   4.74932023 0.97667494 0.76048889 1.4477377  2.18118108
  1.93802882 3.71873769 4.48046453]]


In [ ]:
from stable_baselines3 import DQN

# Assuming the model was saved as "trained_dqn_env_0.zip"
model_path = "trained_dqn.zip"

# Load the model
model = DQN.load(model_path)

# You can now use the loaded model for inference or further training


In [58]:
test1 = CustomFrozenLakeEnv(environments[899],land_data)

In [66]:
environments[899][4][3]

3.9032158851623535

In [69]:
obs = test1.reset()
test1.render()
new_state = 40
print(environments[899][4][4])
# Run the inference loop until the episode is done
while True:
    # Use the model to predict the action given the current observation
    action, _ = model.predict(obs)
    print(action)
    if action == 0:
      new_state-=1
    if action == 1:
      new_state+=1
    if action == 2:
      new_state = (state)*(int((state/9))-1) + state%9
    if action == 3:
      new_state = (state)*(int((state/9))+1) + state%9
    print(environments[899][int(new_state/9)][new_state%9])
    # Take the predicted action and get the next observation, reward, and done flag
    obs, reward, done, _ = test1.step(action)

    # Render the environment to visualize the agent's actions
    test1.render()

    # Check if the episode is done
    if done:
        break

# Close the environment
test1.close()

CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCFHCC
CCCFfCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
4.061974048614502
0
3.9032158851623535
CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCFHCC
CCCfFCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
0
3.4544825553894043
CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCFHCC
CCcFFCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
0
2.9788951873779297
CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCFHCC
CcCFFCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
0
2.630740165710449
CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCFHCC
cCCFFCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
0
3.9253363609313965
CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCFHCc
CCCFFCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
0
3.9844837188720703
CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCFHcC
CCCFFCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
0
4.065456867218018
CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCFhCC
CCCFFCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
[0]
4.065474033355713
CCCFFCCCC
CFCFCFFFC
CFCCCFHCC
CFCCCfHCC
CCCFFCCCC
CFCCCFHCC
CFFCFBHCC
CFFFFFCCC
CFFFCCCCC
[0]
4

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-53-16e60f310ad9>:56: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if int(state) in self.visited:
